In [1]:
import os

In [3]:
%pwd

'd:\\MachineLearning\\Nlp project1\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\MachineLearning\\Nlp project1\\Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
  root_dir:Path
  data_path:Path
  model_ckpt:Path
  num_train_epochs: int
  warmup_steps: int
  per_device_train_batch_size: int
  weight_decay: int
  logging_steps: int
  evaluation_strategy: str
  eval_steps: int
  save_steps: int
  gradient_accumulation_steps: int


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
      config = self.config.model_trainer
      params = self.params.TrTrainingArguments
      
      
      create_directories([config.root_dir])
      
      model_trainer_config = ModelTrainerConfig(
        root_dir=config.root_dir,
        data_path = config.data_path,
        model_ckpt= config.model_ckpt,
        num_train_epochs=params.num_train_epochs,
        warmup_steps=params.warmup_steps,
        per_device_train_batch_size= params.per_device_train_batch_size,
        weight_decay=params.weight_decay,
        logging_steps=params.logging_steps,
        eval_steps=params.eval_steps,
        evaluation_strategy=params.evaluation_strategy,
        save_steps= params.save_steps,
        gradient_accumulation_steps= params.gradient_accumulation_steps
        
        
              )
      return model_trainer_config
    

In [11]:
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from textSummarizer.logging import logger
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch

In [ ]:
class ModelTrainer:
  def __init__(self,config:ModelTrainerConfig):
    self.config = config
    
    
  def train(self):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
    model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
    seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_pegasus)
    
    #loading the data
    dataset_samsum_pt = load_from_disk(self.config.data_path)
    
    trainer_args = TrainingArguments(
      output_dir=self.config.output_dir, num_train_epochs=self.config.num_train_epochs,
           
      
    )
    
    
    
    